## 1. Import các thư viện cần thiết

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.c79RvzvJOt/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.hf70zCkzK7/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.0Alr9pOAi0/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu

In [49]:
import pandas as pd
import os
import requests
import time
import random

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## 2. Crawl

In [2]:
datasets = []

In [81]:
WEBDRIVER_DELAY_TIME_INT = 10

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.implicitly_wait(4)
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INT)

<ipython-input-81-4705f559d4c8>:11: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


In [83]:
deletion_script = 'arguments[0].parentNode.removeChild(arguments[0]);'
for page_idx in tqdm(range(1, 10)):
    main_url = f'https://www.thivien.net/searchpoem.php?PoemType=16&ViewType=1&Country=2&Age[]=3&Page={page_idx}' 
    driver.get(main_url)

    content_tags_xpath = '//*[@id="PageContent"]/div[2]/div/div[@class="list-item"]'
    content_tags = driver.find_elements(By.XPATH, content_tags_xpath)
    for idx in range(len(content_tags)):
        content_tag_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]'
        content_title_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]/h4/a'
        content_tag = wait.until(EC.presence_of_element_located((By.XPATH, content_tag_xpath)))
        poem_title = wait.until(EC.presence_of_element_located((By.XPATH, content_title_xpath))).text
        poem_url = wait.until(EC.presence_of_element_located((By.XPATH, content_title_xpath))).get_attribute('href')

        try:
            driver.get(poem_url) 
            
            poem_src_xpath = '//span[@class="small"]'
            poem_content_tag = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'poem-content')))

            try:
                poem_content_i_tag = poem_content_tag.find_element(By.TAG_NAME, 'i')
                driver.execute_script(deletion_script, poem_content_i_tag)
            except:
                pass

            try:
                poem_content_b_tag = poem_content_tag.find_element(By.TAG_NAME, 'b')
                driver.execute_script(deletion_script, poem_content_b_tag)
            except:
                pass

            poem_content = poem_content_tag.text

            try:
                poem_src_tag = wait.until(EC.presence_of_element_located((By.XPATH, poem_src_xpath)))
                poem_src = poem_src_tag.text
            except:
                poem_src = ''

            poem_info = {
                'title': poem_title,
                'content': poem_content,
                'source': poem_src,
                'url': poem_url
            }

            datasets.append(poem_info)

            driver.back()
        except Exception as e:
            print(e)
            print(poem_url)

100%|██████████| 9/9 [24:53<00:00, 165.98s/it]


In [84]:
datasets

[{'title': '“Cái làm ta hạnh phúc”',
  'content': 'Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều\nChỉ cần có ai đó\nĐể ta thầm thương yêu\n\nRồi thêm chút công việc\nCho ta làm hàng ngày\nCuối cùng, chút mơ mộng\nĐể đưa ta lên mây',
  'source': 'Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB Lao động, 2013',
  'url': 'https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3%A2n/C%C3%A1i-l%C3%A0m-ta-h%E1%BA%A1nh-ph%C3%BAc/poem-t24-M5Dn2cxmEDVSrQhtCw'},
 {'title': '“Chiều vừa xốp trên tay”',
  'content': 'Chiều vừa xốp trên tay\nChợt nghe thoáng ong bay\nCó ai vừa chết nhỉ\nMây thắt tang trăng gầy\n\nỚt đỏ sao cứ đỏ\nTáo chín cho thật vàng\nEm đẹp cho thêm đẹp\nĐể mắt ngừng lang thang',
  'source': 'Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB Văn học, 1999',
  'url': 'https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA%ADn/Chi%E1%BB%81u-v%E1%BB%ABa-x%E1%BB%91p-tr%C3%AAn-tay/poem-f31v-s36HNxJNqauIrC7Xw'},
 {'title': '“Dưới giàn hoa thiên lý...”',
  'content': 'Dưới giàn hoa thiên lý\nMột mình anh đang

In [87]:
df = pd.DataFrame(datasets)
df.to_csv('poem_dataset.csv', index=True)

In [88]:
df

,title,content,source,url
0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...
85,Anh Đáp tự thiêu,Đời anh hăm hai tuổi\nChỉ còn cục than đen.\nC...,"7-1958\n\nNguồn: Huy Cận, Trời mỗi ngày lại sá...",https://www.thivien.net/Huy-C%E1%BA%ADn/Anh-%C...
86,Anh đi công tác về,Anh đi công tác về\nEm chạy ào ra đón\nCả nhà ...,,https://www.thivien.net/L%C3%BD-Ph%C6%B0%C6%A1...
87,Anh đi qua đời em,Anh đi qua đời em\nNhư cơn mưa mùa hè\nTắm mưa...,,https://www.thivien.net/Phi-Tuy%E1%BA%BFt-Ba/A...
88,Anh điên,Anh nằm ngoài sự thực\nEm ngồi trong chiêm bao...,,https://www.thivien.net/H%C3%A0n-M%E1%BA%B7c-T...


In [89]:
!zip -r poem_dataset.zip poem_dataset.csv

  adding: poem_dataset.csv (deflated 61%)
